Published on January 09, 2023. By Marília Prata, mpwolke.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.graph_objs as go
import plotly.offline as py
import plotly.express as px

import dask.dataframe as dd
import dask.array as da
from dask.diagnostics import ProgressBar

#Ignore warnings
import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#Competition Citation

@misc{hms-harmful-brain-activity-classification,

    author = {Jin Jing, Zhen Lin, Chaoqi Yang, Ashley Chow, Sohier Dane, Jimeng Sun, M. Brandon Westover},
    
    title = {HMS - Harmful Brain Activity Classification },
    
    publisher = {Kaggle},
    year = {2024},
    url = {https://kaggle.com/competitions/hms-harmful-brain-activity-classification}

#The Ictal-Interictal Continuum:

![](https://i0.wp.com/emcrit.org/wp-content/uploads/2021/10/Screen-Shot-2021-11-02-at-2.32.31-PM.png?resize=614%2C474&ssl=1)https://emcrit.org/emcrit/eeg-terminology/

"This category includes rhythmic patterns that disrupt the background. They may contribute to poor mental status and perhaps contribute to neuronal injury. The four most mentioned are Generalized Rhythmic Delta Activity (GRDA), Lateralized Rhythmic Delta Activity (LRDA), Generalized Periodic Discharges (GPDs), and Lateralized Periodic Discharges (LPDs). There are others."

"Generalized Rhythmic Delta (GRDA):

"A non-specific pattern that is slow (<5 hz) and may be seen in profound encephalopathy, post-ictally or with inflammatory, degenerative, traumatic, or toxic-metabolic disorders."

Lateralized Rhythmic Delta (LRDA):

"Another lateralized slow (<5hz) pattern that usually reflects the presence of a focal lesion; associated with risk of acute seizures, especially non-convulsive status epilepticus."

Generalized Periodic Discharges (GPDs):

"Sharp waves or spikes that occur rhythmically in the right and left brain.  They may occur infrequently (0.5hz) or quite frequently (2hz). Tend to be seen in diffuse processes: toxic-metabolic encephalopathy, sepsis.  But may also be seen in herpes simplex virus encephalitis and autoimmune encephalopathies. Can be associated with NCSE."

Lateralized Periodic Discharges (LPDs):

"Lateralized sharp waves or spikes which made have associated slow waves.  Also can be seen infrequently or frequently (2hz).  Commonly encountered in stroke, intracerebral hemorrhage, subarachnoid hemorrhage, tumors, abscesses, Creutzfeldt-Jakob disease, herpes simplex virus and other infectious/autoimmune pathology.  LPDs are highly associated with seizures, especially in the setting of acute illness, metabolic disturbances or focal lesions."

Citation: NeuroEMCrit Team (Casey & Neha). NeuroEMCrit – Demystifying the EEG Report. EMCrit Blog. Published on November 11, 2021. Accessed on January 9th 2024. Available at [https://emcrit.org/emcrit/eeg-terminology/ ].

In [ ]:
train = pd.read_csv('/kaggle/input/hms-harmful-brain-activity-classification/train.csv')
train.head()

#ACNS Critical Care EEG Terminology 2021

[American Clinical Neurophysiology Society’s Standardized
Critical Care EEG Terminology: 2021 Version](https://www.acns.org/UserFiles/file/ACNSStandardizedCriticalCareEEGTerminology_rev2021.pdf)

2021 ACNS Critical Care EEG Terminology 

CONTENTS

* EEG BACKGROUND
*  SPORADIC EPILEPTIFORM DISCHARGES
*  RHYTHMIC AND PERIODIC PATTERNS (RPPs)
*  ELECTROGRAPHIC AND ELECTROCLINICAL SEIZURES [NEW, 2021]
*  BRIEF POTENTIALLY ICTAL RHYTHMIC DISCHARGES (BIRDs) [NEW, 2021]
*  ICTAL-INTERICTAL CONTINUUM (IIC) [NEW, 2021]
*  MINIMUM REPORTING REQUIREMENTS
*  OTHER TERMS

ABBREVIATION LIST

ACNS ¼ American Clinical Neurophysiology Society

BI ¼ Bilateral Independent

BIRDs ¼ Brief Potentially Ictal Rhythmic Discharges

BTC ¼ Bilateral Tonic-Clonic

CAPE ¼ Cyclic Alternating Pattern of Encephalopathy

CCEMRC ¼ Critical Care EEG Monitoring Research

Consortium

ECSz ¼ Electroclinical seizure

ECSE ¼ Electroclinical status epilepticus

EDs ¼ Epileptiform Discharges

EDB ¼ Extreme Delta Brush

EEG ¼ Electroencephalography

ESE ¼ Electrographic status epilepticus

ESz ¼ Electrographic seizure

G ¼ Generalized

GPFA ¼ Generalized Paroxysmal Fast Activity

Hz ¼ Hertz (i.e., per second)

IIC ¼ Ictal-Interictal Continuum

L ¼ Lateralized

Mf ¼ Multifocal

PDs ¼ Periodic Discharges

RDA ¼ Rhythmic Delta Activity

RPP ¼ Rhythmic or Periodic Pattern (i.e., PDs, RDA or SW)

SE ¼ Status epilepticus

SI ¼ Stimulus-Induced.

SIRPIDs ¼ Stimulus-Induced Rhythmic, Periodic or IctalAppearing Discharges

ST ¼ Stimulus-Terminated

SW ¼ Spike-and-wave or sharp-and-wave

UI ¼ Unilateral Independent

1 ¼ Plus ¼ Additional feature which renders the pattern

more ictal-appearing than the usual term without the plus

1F ¼ Superimposed fast activity

1R ¼ Superimposed rhythmic activity

1S ¼ Superimposed sharp waves or spikes, or sharply contoured

https://www.acns.org/UserFiles/file/ACNSStandardizedCriticalCareEEGTerminology_rev2021.pdf

In [ ]:
test = pd.read_csv('/kaggle/input/hms-harmful-brain-activity-classification/test.csv')
test.head()

In [ ]:
sub = pd.read_csv('/kaggle/input/hms-harmful-brain-activity-classification/sample_submission.csv')
sub.head()

#Read one electroencephalography (EEG) parquet with Dask

In [ ]:
ddf = dd.read_parquet("/kaggle/input/hms-harmful-brain-activity-classification/train_eegs/1712674008.parquet")

In [ ]:
%%time
len(ddf)

In [ ]:
ddf.head()

In [ ]:
ddf.EKG.describe().compute()

In [ ]:
ddf.Fp1.describe().compute()

#Spectogram Parquet file - Reading them without Dask

In [ ]:
spec_train = pd.read_parquet('../input/hms-harmful-brain-activity-classification/train_spectrograms/1000086677.parquet')

In [ ]:
spec_train.head()

In [ ]:
#3rd row, 3rd colunm

spec_train.iloc[2,2]

#Almost 3 hours trying to find out to deliver a Spectogram with Parquet Files: TY Corey Lehmann

I've alredy learned spectograms with wav/audio/hdf5/oog files though I was suffering to plot with Parquet files.  Anyway, my head is acking so I've just copied Corey's Lehmann snippets. And changed the cmap to Spectral, In fact, I should have applied Inferno, cause that's how I'm felling now. Please vote Corey's:

https://www.kaggle.com/code/clehmann10/plot-spectrograms/notebook

In [ ]:
#By Corey Lehmann https://www.kaggle.com/code/clehmann10/plot-spectrograms/notebook

comp_path = '/kaggle/input/hms-harmful-brain-activity-classification'
train_spect_dir = '/'.join([comp_path, 'train_spectrograms'])
train_spect_path_list = [entry.path for entry in os.scandir(train_spect_dir)]

In [ ]:
#By Corey Lehmann https://www.kaggle.com/code/clehmann10/plot-spectrograms/notebook

def plot_spectrogram(spectrogram_path):
    sample_spect = pd.read_parquet(spectrogram_path)
    
    split_spect = {
        "LL": sample_spect.filter(regex='^LL', axis=1),
        "RL": sample_spect.filter(regex='^RL', axis=1),
        "RP": sample_spect.filter(regex='^RP', axis=1),
        "LP": sample_spect.filter(regex='^LP', axis=1),
    }
    
    fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(15, 12))
    axes = axes.flatten()
    label_interval = 5
    for i, split_name in enumerate(split_spect.keys()):
        ax = axes[i]
        img = ax.imshow(np.log(split_spect[split_name]).T, cmap='Spectral', aspect='auto', origin='lower')  # You can choose any colormap (cmap) that suits your preferences
        cbar = fig.colorbar(img, ax=ax)
        cbar.set_label('Log(Value)')
        ax.set_title(split_name)
        ax.set_ylabel("Frequency (Hz)")
        ax.set_xlabel("Time")

        ax.set_yticks(np.arange(len(split_spect[split_name].columns)))
        ax.set_yticklabels([column_name[3:] for column_name in split_spect[split_name].columns])
        frequencies = [column_name[3:] for column_name in split_spect[split_name].columns]
        ax.set_yticks(np.arange(0, len(split_spect[split_name].columns), label_interval))
        ax.set_yticklabels(frequencies[::label_interval])
    plt.tight_layout()
    plt.show()
    
plot_spectrogram(train_spect_path_list[0])

Another Message: "It looks like you ran out of disk space. Upgrade to run on larger machines with more memory. You can export your notebook to get started."

#Seizures Competitions on Kaggle

[UPenn and Mayo Clinic's Seizure Detection Challenge](https://www.kaggle.com/competitions/seizure-detection)

[American Epilepsy Society Seizure Prediction Challenge](https://www.kaggle.com/competitions/seizure-prediction)

[Melbourne University AES/MathWorks/NIH Seizure Prediction](https://www.kaggle.com/competitions/melbourne-university-seizure-prediction)

[Cornell ECE/BME 5040 Project](https://www.kaggle.com/competitions/cornell-ecebme-5040-project)

[EEG Seizure Analysis](https://www.kaggle.com/competitions/eeg-seizure-analysis)

#Acknowledgements:

Corey Lehmann https://www.kaggle.com/code/clehmann10/plot-spectrograms/notebook